In [1]:
import numpy as np
import pandas as pd
import queue
import matplotlib.pyplot as plt

from BayesianOptimization.bayesian_optimization import BayesianOptimization

from Backtest.backtest import Backtest
from Backtest.data import OHLCDataHandler
from BOLLStrategy import BOLLStrategy
from Backtest.open_json_gz_files import open_json_gz_files
from Backtest.generate_bars import generate_bars

In [2]:
def run_backtest(config, trading_data, ohlc_data, window, a):
    window = int(window)
    config['title'] = "BOLLStrategy" + "_" + str(window) + "_" + str(a)
    print("---------------------------------")
    print(config['title'])
    print("---------------------------------")

    events_queue = queue.Queue()

    data_handler = OHLCDataHandler(
        config, events_queue,
        trading_data=trading_data, ohlc_data=ohlc_data
    )
    strategy = BOLLStrategy(config, events_queue, data_handler,
                            window = window, a = a)

    backtest = Backtest(config, events_queue, strategy,
                        data_handler=data_handler)

    results = backtest.start_trading()

    # dict_ans = {
    #     "window": [window],
    #     "a": [a],
    #     "Sharpe Ratio": [results['sharpe']],
    #     "Total Returns": [(results['cum_returns'][-1] - 1)],
    #     "Max Drawdown": [(results["max_drawdown"] * 100.0)],
    #     "Max Drawdown Duration": [(results['max_drawdown_duration'])],
    #     "Trades": [results['trade_info']['trading_num']],
    #     "Trade Winning": [results['trade_info']['win_pct']],
    #     "Average Trade": [results['trade_info']['avg_trd_pct']],
    #     "Average Win": [results['trade_info']['avg_win_pct']],
    #     "Average Loss": [results['trade_info']['avg_loss_pct']],
    #     "Best Trade": [results['trade_info']['max_win_pct']],
    #     "Worst Trade": [results['trade_info']['max_loss_pct']],
    #     "Worst Trade Date": [results['trade_info']['max_loss_dt']],
    #     "Avg Days in Trade": [results['trade_info']['avg_dit']]
    # }
    # return pd.DataFrame(dict_ans)
    return (results['cum_returns'][-1] - 1)

## Setting

In [3]:
config = {
    "csv_dir": "C:/backtest/Binance",
    "out_dir": "C:/backtest/results/BOLLStrategy",
    "title": "BOLLStrategy",
    "is_plot": False,
    "save_plot": False,
    "save_tradelog": False,
    "start_date": pd.Timestamp("2017-07-01T00:0:00", freq="60" + "T"),  # str(freq) + "T"
    "end_date": pd.Timestamp("2018-09-01T00:00:00", freq="60" + "T"),
    "equity": 1.0,
    "freq": 60,  # min
    "commission_ratio": 0.001,
    "suggested_quantity": None,  # None or a value
    "max_quantity": None,  # None or a value, Maximum purchase quantity
    "min_quantity": None,  # None or a value, Minimum purchase quantity
    "min_handheld_cash": None,  # None or a value, Minimum handheld funds
    "exchange": "Binance",
    "tickers": ['BTCUSDT']
}

In [4]:
# trading_data = {}
# for ticker in config['tickers']:
#     # trading_data[ticker] = open_gz_files(config['csv_dir'], ticker)
#     trading_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker + '.h5', key=ticker)

ohlc_data = {}
for ticker in config['tickers']:
    # ohlc_data[ticker] = generate_bars(trading_data, ticker, config['freq'])
    ohlc_data[ticker] = pd.read_hdf(config['csv_dir'] + '\\' + ticker +'_OHLC_60min.h5', key=ticker)

trading_data = None

In [5]:
gp_params = {"alpha": 1e-5}

## Acquisition Function "Upper Confidence Bound"

* Prefer exploitation (kappa=1.0)

* Prefer exploration (kappa=10)


In [6]:
BO = BayesianOptimization(
    run_backtest,
    pbounds = {'window': (1, 240),
                'a': (0.1, 3)},
    is_int=[1, 0],
    invariant = {
        'config': config,
        'trading_data': trading_data,
        'ohlc_data': ohlc_data
    },
    random_state = 1
)
BO.explore({
        'window': np.arange(1, 240, 24),
        'a': np.arange(0.1, 3, 0.3)
    },
    eager=True)
BO.maximize(init_points=0, n_iter=10, acq='ucb', kappa=5, **gp_params)

---------------------------------
BOLLStrategy_1_0.1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: nan
Max Drawdown: 0.0000000000
Max Drawdown Duration: 0
Total Returns: 0.0000000000
---------------------------------
Trades: 0
Trade Winning: N/A
Average Trade: N/A
Average Win: N/A
Average Loss: N/A
Best Trade: N/A
Worst Trade: N/A
Worst Trade Date: 0
Avg Days in Trade: 0
---------------------------------
    1 | 00m02s |    0.00000 | 0.100000 | 1.000000 | 
---------------------------------
BOLLStrategy_25_0.4
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Running Backtest...
------------------------

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.1309545969
Max Drawdown: 53.4794344911
Max Drawdown Duration: 5830
Total Returns: 0.5624907279
---------------------------------
Trades: 79
Trade Winning: 25%
Average Trade: 1.03%
Average Win: 10.42%
Average Loss: -2.15%
Best Trade: 36.97%
Worst Trade: -9.40%
Worst Trade Date: 1 days 12:00:00
Avg Days in Trade: 1 days 21:40:15
---------------------------------
   11 | 00m13s |    0.56249 | 0.100000 | 132.000000 | 
---------------------------------
BOLLStrategy_180_0.1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.6236176503
Max Drawdown: 55.0785667380
Max Drawdown Duration: 5604
Total Returns: 1.0583022310
------

In [7]:
print(BO.res['max'])

{'max_val': 1.5564147596551323, 'max_params': {'window': 121.0, 'a': 1.6000000000000003}}


In [8]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
Target.to_csv(config['out_dir'] + "/target_ucb.csv")
Target.sort_values(by = "Target")

,Parameters,Target
18,"[9.0, 0.1]",-0.821221
0,"[1.0, 0.1]",0.000000
9,"[217.0, 2.8000000000000003]",0.069652
17,"[110.0, 3.0]",0.207846
14,"[66.0, 0.1]",0.250939
1,"[25.0, 0.4]",0.360754
4,"[97.0, 1.3000000000000003]",0.422273
10,"[132.0, 0.1]",0.562491
13,"[60.0, 3.0]",0.576544
8,"[193.0, 2.5000000000000004]",0.796573


## Acquisition Function "Expected Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)

In [13]:
BO = BayesianOptimization(
    run_backtest,
    pbounds = {'window': (1, 240),
                'a': (0.1, 3)},
    is_int=[1, 0],
    invariant = {
        'config': config,
        'trading_data': trading_data,
        'ohlc_data': ohlc_data
    },
    random_state = 1
)
BO.explore({
        'window': np.arange(1, 240, 24),
        'a': np.arange(0.1, 3, 0.3)
    },
    eager=True)
BO.maximize(init_points=0, n_iter=10, acq="ei", xi=0.01, **gp_params)

---------------------------------
ADXStrategy_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -6.9007275153
Max Drawdown: 98.8338690249
Max Drawdown Duration: 6661
Total Returns: -0.9866560406
---------------------------------
Trades: 1231
Trade Winning: 38%
Average Trade: -0.13%
Average Win: 1.35%
Average Loss: -1.04%
Best Trade: 11.71%
Worst Trade: -7.69%
Worst Trade Date: 0 days 01:00:00
Avg Days in Trade: 0 days 02:49:49
---------------------------------
    1 | 00m04s |   -0.98666 | 1.000000 | 
---------------------------------
ADXStrategy_21
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Runni

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.2138615878
Max Drawdown: 61.4626828553
Max Drawdown Duration: 5830
Total Returns: -0.0525022310
---------------------------------
Trades: 68
Trade Winning: 43%
Average Trade: 0.35%
Average Win: 4.51%
Average Loss: -2.75%
Best Trade: 39.17%
Worst Trade: -15.78%
Worst Trade Date: 1 days 07:00:00
Avg Days in Trade: 1 days 14:11:28
---------------------------------
   12 | 00m05s |   -0.05250 | 221.000000 | 
Initialization
-----------------------------------------
 Step |   Time |      Value |    window | 
Bayesian Optimization
-----------------------------------------
 Step |   Time |      Value |    window | 
---------------------------------
ADXStrategy_22
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Running Backtest...
---------------------------------
-------------

In [14]:
print(BO.res['max'])

{'max_val': 0.9908953915556202, 'max_params': {'window': 21.0}}


In [15]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
# Target.to_csv(config['out_dir'] + "/target_ei.csv")
Target.sort_values(by = "Target")

,Parameters,Target
0,[1.0],-0.986656
19,[4.0],-0.927194
11,[221.0],-0.052502
14,[38.0],-0.018888
18,[190.0],0.051317
4,[81.0],0.093811
15,[149.0],0.107946
6,[121.0],0.115519
2,[41.0],0.144393
20,[115.0],0.173126


In [17]:
BO.maximize(init_points=0, n_iter=10, acq="ei", xi=0.0001, **gp_params)

Bayesian Optimization
-----------------------------------------
 Step |   Time |      Value |    window | 
---------------------------------
ADXStrategy_70
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.6488930882
Max Drawdown: 34.2953464468
Max Drawdown Duration: 5830
Total Returns: 1.0160987241
---------------------------------
Trades: 119
Trade Winning: 38%
Average Trade: 0.92%
Average Win: 4.89%
Average Loss: -1.49%
Best Trade: 27.64%
Worst Trade: -9.62%
Worst Trade Date: 1 days 17:00:00
Avg Days in Trade: 1 days 00:38:49
---------------------------------
   23 | 00m06s |    1.01610 | 70.000000 | 
---------------------------------
ADXStrategy_133
---------------------------------
Data Time Interval for BTCUSDT:

In [18]:
print(BO.res['max'])

{'max_val': 1.0160987241398645, 'max_params': {'window': 70.0}}


In [23]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
Target.to_csv(config['out_dir'] + "/target_ei.csv")
Target.sort_values(by = "Target")

,Parameters,Target
0,[1.0],-0.986656
19,[11.0],-0.421099
11,[221.0],-0.052502
4,[81.0],0.093811
6,[121.0],0.115519
2,[41.0],0.144393
16,[138.0],0.184027
10,[201.0],0.201388
18,[48.0],0.210086
15,[144.0],0.232713


## Acquisition Function "Probability of Improvement"

* Prefer exploitation (xi=1e-4)
* Prefer exploration (xi=0.1)

In [21]:
BO = BayesianOptimization(
    run_backtest,
    pbounds = {'window': (1, 240),
                'a': (0.1, 3)},
    is_int=[1, 0],
    invariant = {
        'config': config,
        'trading_data': trading_data,
        'ohlc_data': ohlc_data
    },
    random_state = 1
)
BO.explore({
        'window': np.arange(1, 240, 24),
        'a': np.arange(0.1, 3, 0.3)
    },
    eager=True)
BO.maximize(init_points=0, n_iter=10, acq="poi", xi=0.01, **gp_params)

---------------------------------
ADXStrategy_1
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: -6.9007275153
Max Drawdown: 98.8338690249
Max Drawdown Duration: 6661
Total Returns: -0.9866560406
---------------------------------
Trades: 1231
Trade Winning: 38%
Average Trade: -0.13%
Average Win: 1.35%
Average Loss: -1.04%
Best Trade: 11.71%
Worst Trade: -7.69%
Worst Trade Date: 0 days 01:00:00
Avg Days in Trade: 0 days 02:49:49
---------------------------------
    1 | 00m05s |   -0.98666 | 1.000000 | 
---------------------------------
ADXStrategy_21
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Runni

---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 0.2138615878
Max Drawdown: 61.4626828553
Max Drawdown Duration: 5830
Total Returns: -0.0525022310
---------------------------------
Trades: 68
Trade Winning: 43%
Average Trade: 0.35%
Average Win: 4.51%
Average Loss: -2.75%
Best Trade: 39.17%
Worst Trade: -15.78%
Worst Trade Date: 1 days 07:00:00
Avg Days in Trade: 1 days 14:11:28
---------------------------------
   12 | 00m04s |   -0.05250 | 221.000000 | 
Initialization
-----------------------------------------
 Step |   Time |      Value |    window | 
Bayesian Optimization
-----------------------------------------
 Step |   Time |      Value |    window | 
---------------------------------
ADXStrategy_22
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Running Backtest...
---------------------------------
-------------

In [22]:
print(BO.res['max'])

{'max_val': 0.9908953915556202, 'max_params': {'window': 21.0}}


In [24]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
# Target.to_csv(config['out_dir'] + "/target_pi.csv")
Target.sort_values(by = "Target")

,Parameters,Target
0,[1.0],-0.986656
19,[11.0],-0.421099
11,[221.0],-0.052502
4,[81.0],0.093811
6,[121.0],0.115519
2,[41.0],0.144393
16,[138.0],0.184027
10,[201.0],0.201388
18,[48.0],0.210086
15,[144.0],0.232713


In [26]:
BO.maximize(init_points=0, n_iter=10, acq="poi", xi=0.0001, **gp_params)

Bayesian Optimization
-----------------------------------------
 Step |   Time |      Value |    window | 
---------------------------------
ADXStrategy_69
---------------------------------
Data Time Interval for BTCUSDT:
	Start Date	: 2017-10-27 22:00:00
	End Date	: 2018-08-07 23:00:00
---------------------------------
Running Backtest...
---------------------------------
---------------------------------
Backtest complete.
---------------------------------
Sharpe Ratio: 1.2325719271
Max Drawdown: 41.3920074235
Max Drawdown Duration: 5830
Total Returns: 0.6258643073
---------------------------------
Trades: 132
Trade Winning: 33%
Average Trade: 0.69%
Average Win: 4.85%
Average Loss: -1.39%
Best Trade: 27.78%
Worst Trade: -10.13%
Worst Trade Date: 1 days 16:00:00
Avg Days in Trade: 0 days 22:25:00
---------------------------------
   23 | 00m06s |    0.62586 | 69.000000 | 
---------------------------------
ADXStrategy_208
---------------------------------
Data Time Interval for BTCUSDT

In [27]:
print(BO.res['max'])

{'max_val': 0.9908953915556202, 'max_params': {'window': 21.0}}


In [28]:
Target = pd.DataFrame({'Parameters': BO.X.tolist(), 'Target': BO.Y})
Target.to_csv(config['out_dir'] + "/target_pi.csv")
Target.sort_values(by = "Target")

,Parameters,Target
0,[1.0],-0.986656
19,[11.0],-0.421099
11,[221.0],-0.052502
24,[38.0],-0.018888
23,[208.0],-0.017305
27,[204.0],0.026351
29,[191.0],0.034829
4,[81.0],0.093811
6,[121.0],0.115519
28,[30.0],0.134312
